# MLOps 구축하기
--------------

## 1. SageMaker Studio 에서 Pipeline 실행

사용 중인 region에서 SageMaker Studio를 실행합니다. Event Engine을 사용하시는 분들은 이미 SageMaker Studio가 생성되어 있기 때문에 AWS 콘솔에서 SageMaker에 있는 Studio를 클릭하여 **open Studio**를 클릭하여 환경으로 접속하게 됩니다.

<p align="center">
<center><img src="./img/studio_notebook.png" height="250" width="850" alt=""><center>
<br><br>
</p>
    
SageMaker studio에 접속한 다음, SageMaker의 Pipeline을 생성하기 위해 Project를 생성합니다. **create project**를 선택한 다음 **MLOps template for model building and training**을 선택한 다음 **select project template**을 생성합니다.
<p align="center">
<center><img src="./img/project_create.png" height="250" width="850" alt=""><center>
<br><br>
</p>   
    
원하시는 project 이름을 입력한 다음, **create project**를 클릭하여 project를 생성합니다.
<p align="center">
<center><img src="./img/project_create_detail.png" height="150" width="550" alt=""><center>
<br><br>
</p>   

In [ ]:
import boto3
import json
from sagemaker import get_execution_role
from time import strftime
import calendar
import time

In [ ]:
%store -r
print(f"default_bucket : {default_bucket}")

### 1. 활용할 소스코드를 CodeCommit에 업로드 하기

데이터 과학자가 개발한 소스코드를 S3 또는 CodeCommit, Github 등을 통해 플랫폼 엔지니어가 개발하는 플랫폼으로 전달할 수 있습니다.

In [ ]:
iam_client = boto3.client('iam')

In [ ]:
iam_client.attach_role_policy(
    RoleName='AmazonSageMakerServiceCatalogProductsUseRole',
    PolicyArn='arn:aws:iam::aws:policy/SecretsManagerReadWrite'
)

In [ ]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()['Account']

In [ ]:
codecommit_client = boto3.client('codecommit')

In [ ]:
rep_name = codecommit_client.list_repositories(
    sortBy='lastModifiedDate',
    order='descending' ## 'descending' ascending
)

for rep_item in rep_name['repositories']:
    if 'informer-' in rep_item['repositoryName']:
        repositoryName = rep_item['repositoryName']
    elif 'informer2020' in rep_item['repositoryName']:
        code_repositoryName = rep_item['repositoryName']

        
# repositoryName=rep_name['repositories'][0]['repositoryName']

response = codecommit_client.get_repository(
    repositoryName=repositoryName
)

code_response = codecommit_client.get_repository(
    repositoryName=code_repositoryName
)

In [ ]:
pipeline_giturl = response['repositoryMetadata']['cloneUrlHttp']
code_commit_repo = code_response['repositoryMetadata']['cloneUrlHttp']
print(f"pipeline_giturl : {pipeline_giturl}\ncode_commit_repo : {code_commit_repo}")

In [ ]:
!git clone $pipeline_giturl

In [ ]:
sm_client = boto3.client("sagemaker")

In [ ]:
res_model_pkg = sm_client.list_model_package_groups(
    SortBy='CreationTime',
    SortOrder='Descending'
)

In [ ]:
model_package_group_name = res_model_pkg['ModelPackageGroupSummaryList'][0]['ModelPackageGroupName']
model_package_group_name

In [ ]:
%%writefile ./pipelines/informer/pipeline.py
"""Example workflow pipeline script for abalone pipeline.

                                               . -RegisterModel
                                              .
    Process-> Train -> Evaluate -> Condition .
                                              .
                                               . -(stop)

Implements a get_pipeline(**kwargs) method.
"""

# sagemaker-experiments

import os

import boto3
import sagemaker
import sagemaker.session

import datetime
import glob
import os
import time
import warnings

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial


import shutil

import boto3
import numpy as np
import pandas as pd
import subprocess
import json

# from tqdm import tqdm
from time import strftime

from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

from sagemaker.inputs import TrainingInput

from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.processing import FrameworkProcessor
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import CacheConfig

BASE_DIR = os.path.dirname(os.path.realpath(__file__))
SOURCE_DIR = 'Informer2020'


def get_sagemaker_client(region):
    """Gets the sagemaker client.

        Args:
            region: the aws region to start the session
            default_bucket: the bucket to use for storing the artifacts

        Returns:
            `sagemaker.session.Session instance
    """
    boto_session = boto3.Session(region_name=region)
    sagemaker_client = boto_session.client("sagemaker")
    return sagemaker_client


def get_session(region):
    """Gets the sagemaker session based on the region.

    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        `sagemaker.session.Session instance
    """

    boto_session = boto3.Session(region_name=region)

    sagemaker_client = boto_session.client("sagemaker")
    runtime_client = boto_session.client("sagemaker-runtime")
    return sagemaker.session.Session(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        sagemaker_runtime_client=runtime_client,
#         default_bucket=default_bucket,
    )

def get_pipeline_custom_tags(new_tags, region, sagemaker_project_arn=None):
    try:
        sm_client = get_sagemaker_client(region)
        response = sm_client.list_tags(
            ResourceArn=sagemaker_project_arn)
        project_tags = response["Tags"]
        for project_tag in project_tags:
            new_tags.append(project_tag)
    except Exception as e:
        print(f"Error getting project tags: {e}")
    return new_tags

###############################################################
###############################################################
### 2번 노트북 - '4. Experments 관리' 부터 복사해서 붙여넣기를 합니다. ###
###############################################################
###############################################################

def get_pipeline(
    region,
    sagemaker_project_arn=None,
    role=None,
    default_bucket=None,
    model_package_group_name="TimeSeries-Group",
    pipeline_name="InformerPipeline",
    base_job_prefix="informer",
):
    """Gets a SageMaker ML Pipeline instance working with on ts data.

    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        an instance of a pipeline
    """
    sagemaker_session = get_session(region)
    default_bucket = sagemaker_session.default_bucket()
    if role is None:
        role = sagemaker.session.get_execution_role(sagemaker_session)

    ### 4. Experiments 관리    
    def create_experiment(experiment_name):
        try:
            sm_experiment = Experiment.load(experiment_name)
        except:
            sm_experiment = Experiment.create(experiment_name=experiment_name,
                                              tags=[{'Key': 'modelname', 'Value': 'informer'}])


    def create_trial(experiment_name, i_type, i_cnt, spot=False):
        create_date = strftime("%m%d-%H%M%s")
        algo = 'informer'

        spot = 's' if spot else 'd'
        i_type = i_type[3:9].replace('.','-')

        trial = "-".join([i_type,str(i_cnt),algo, spot])

        sm_trial = Trial.create(trial_name=f'{experiment_name}-{trial}-{create_date}',
                                experiment_name=experiment_name)

        job_name = f'{sm_trial.trial_name}'
        return job_name

    
    ### 5. 실험 설정
    code_location = f's3://{default_bucket}/poc_informer/sm_codes'
    output_path = f's3://{default_bucket}/poc_informer/output'         
        
        
    metric_definitions = [
        {'Name': 'Epoch', 'Regex': 'Epoch: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
        {'Name': 'train_loss', 'Regex': 'Train Loss: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
        {'Name': 'valid_loss', 'Regex': 'Valid Loss: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
        {'Name': 'test_loss', 'Regex': 'Test Loss: ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?),'},
    ]        
        
    hyperparameters = {
            'model' : 'informer', # model of experiment, options: [informer, informerstack, informerlight(TBD)]
            'data' : 'ETTh1', # data
            'root_path' : 'ETT-small/', # root path of data file
            'data_path' : 'ETTh1.csv', # data file
            'features' : 'M', # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
            'target' : 'OT', # target feature in S or MS task
            'freq' : 'h', # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
            'checkpoints' : 'informer_checkpoints', # location of model checkpoints

            'seq_len' : 96, # input sequence length of Informer encoder
            'label_len' : 48, # start token length of Informer decoder
            'pred_len' : 24, # prediction sequence length
            # Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

            'enc_in' : 7, # encoder input size
            'dec_in' : 7, # decoder input size
            'c_out' : 7, # output size
            'factor' : 5, # probsparse attn factor
            'd_model' : 512, # dimension of model
            'n_heads' : 8, # num of heads
            'e_layers' : 2, # num of encoder layers
            'd_layers' : 1, # num of decoder layers
            'd_ff' : 2048, # dimension of fcn in model
            'dropout' : 0.05, # dropout
            'attn' : 'prob', # attention used in encoder, options:[prob, full]
            'embed' : 'timeF', # time features encoding, options:[timeF, fixed, learned]
            'activation' : 'gelu', # activation
            'distil' : True, # whether to use distilling in encoder
            'output_attention' : False, # whether to output attention in ecoder
            'mix' : True,
            'padding' : 0,
            'freq' : 'h',
            'do_predict' : True,
            'batch_size' : 32,
            'learning_rate' : 0.0001,
            'loss' : 'mse',
            'lradj' : 'type1',
            'use_amp' : False, # whether to use automatic mixed precision training

            'num_workers' : 0,
            'itr' : 1,
            'train_epochs' : 1,  ## Training epochs
            'patience' : 3,
            'des' : 'exp',
            'use_multi_gpu' : True
        }        
     
    experiment_name = 'informer-poc-exp1'                                                          ### <== 1. Experiments 이름 수정
    distribution = None
    do_spot_training = True
    max_wait = None
    max_run = 1*30*60
        
    instance_type="ml.m5.xlarge"                                                                   
    instance_count=1        
    
    
    ### 6. Pipeline parameters, checkpoints와 데이터 위치 설정
    #### 6-1. Pipeline parameters
    train_instance_param = ParameterString(
        name="TrainingInstance",
        default_value="ml.c5.4xlarge",                                                             ### <== 2. Instance 타입, 개수 수정
    )

    train_count_param = ParameterInteger(
        name="TrainingInstanceCount",
        default_value=1
    )

    model_approval_status = ParameterString(
        name="ModelApprovalStatus", default_value="PendingManualApproval"
    )
    
    #### 6-2. checkpoints와 데이터 위치 설정
    image_uri = None
    train_job_name = 'informer'

    if do_spot_training:
        max_wait = max_run

    print("train_job_name : {} \ntrain_instance_type : {} \ntrain_instance_count : {} \nimage_uri : {} \ndistribution : {}".format(train_job_name, train_instance_param.default_value, train_count_param.default_value, image_uri, distribution))    

    
    prefix = 'ETDataset'
    inputs = f's3://{default_bucket}/dataset/{prefix}'

    source_dir = 'Informer2020'                                                                    ### <== 3. git repository내의 소스 코드 위치 
    checkpoint_s3_uri = f's3://{default_bucket}/poc_informer/checkpoints'      
    
    
    #### 6-3. Git 설정 (Secret Manager 활용)
    def get_secret(secret_name):
        secret = {}
        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager'
        )

        # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
        # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        # We rethrow the exception by default.

        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )

        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            secret = json.loads(secret)
        else:
            print("secret is not defined. Checking the Secrets Manager")

        return secret        
        
    ### CodeCommit의 Credentials이 저장된 secret_name 사용이 필요합니다.
    sec_client = boto3.client('secretsmanager')
    secret_name = sec_client.list_secrets(SortOrder='desc')['SecretList'][0]['ARN']               ### <== 4. git credentials이 있는 secret manage 이름 수정
     
    secret=get_secret(secret_name)

    git_config = {'repo': 'https://git-codecommit.${region}.amazonaws.com/v1/repos/informer2020', ### <== 5. git repository 위치 수정
                  'branch': 'main',
                  'username': secret['username'],
                  'password': secret['password']}
        
        
    ### 7. 학습을 위한 Estimator 선언
    create_experiment(experiment_name)
    job_name = create_trial(experiment_name, instance_type, instance_count, spot=do_spot_training)


    estimator = PyTorch(
        entry_point='main_informer.py',
        source_dir=source_dir,
        git_config=git_config,
        role=role,
        sagemaker_session=sagemaker_session,
        framework_version='1.10',
        py_version='py38',
        instance_count=train_count_param,    ## Parameter 값으로 변경
        instance_type=train_instance_param,  ## Parameter 값으로 변경
        volume_size=256,
        code_location = code_location,
        output_path=output_path,
        hyperparameters=hyperparameters,
        distribution=distribution,
        metric_definitions=metric_definitions,
        max_run=max_run,
        checkpoint_s3_uri=checkpoint_s3_uri,
        use_spot_instances=do_spot_training,  # spot instance 활용
        max_wait=max_wait,
        base_job_name=f"training-{job_name}",
        disable_profiler=True,
        debugger_hook_config=False,
    )
    
    
    ### 8. Training 단계 선언    
    from sagemaker.workflow.steps import CacheConfig

    cache_config = CacheConfig(enable_caching=True, 
                               expire_after="7d")        
        
        
    training_step = TrainingStep(
        name="InformerTrain",
        estimator=estimator,
        inputs={
            "training": sagemaker.inputs.TrainingInput(
                s3_data=inputs
            )
        },
        cache_config=cache_config
    )        
        
    
    ### 9. Evaluation 단계 - output에서 압축풀어 test_report.json 가져오기
    framework_processor = FrameworkProcessor(
        PyTorch,
        framework_version="1.10",
        py_version='py38',
        role=role,
        instance_count=1,
        instance_type="ml.c4.xlarge",
        code_location=code_location,
        base_job_name=f"generatingreport-{job_name}",  # choose any name
    )     
        
    model_input = ProcessingInput(
        source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
        destination="/opt/ml/processing/model",
    )      
        
    test_report = PropertyFile(
        name="TestReport",
        output_name="result",
        path="test_report.json",
    )
        
    run_args = framework_processor.get_run_args(
        code="postprocess.py",
        source_dir="Informer2020",
        git_config=git_config,
        inputs=[model_input],
        outputs=[
            ProcessingOutput(output_name="result", source="/opt/ml/processing/result")
        ],
        job_name=f"process-step-{job_name}"
    )        
        

    postprocessing_step = ProcessingStep(
        name="PostProcessingforInformer",  # choose any name
        processor=framework_processor,
        inputs=run_args.inputs,
        outputs=run_args.outputs,
        code=run_args.code,
        property_files=[test_report],
        cache_config=cache_config
    )
        

    ### 10. Model 등록 단계
    model_package_group_name = 'mlops-test-informer-p-XXXXXX'                                           ### <== 6. model package group 이름 수정
    
    model_metrics = ModelMetrics(
        model_statistics=MetricsSource(
            s3_uri="{}/test_report.json".format(
                postprocessing_step.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"],
            ),
            content_type="application/json",
        )
    )
    
    register_step = RegisterModel(
        name="InformerRegisterModel",
        estimator=estimator,
        model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.xlarge"],
        transform_instances=["ml.m5.xlarge"],
        model_package_group_name=model_package_group_name,
        approval_status=model_approval_status,
        model_metrics=model_metrics,
    )    

    
    ### 11. Condition 단계
    cond_lte = ConditionLessThanOrEqualTo(  # You can change the condition here
        left=JsonGet(
            step=postprocessing_step,
            property_file=test_report,
            json_path="regression_metrics.mse.value",  # This should follow the structure of your report_dict defined in the postprocess.py file.
        ),
        right=1.0,  # You can change the threshold here
    )
    cond_step = ConditionStep(
        name="TestMSECond",
        conditions=[cond_lte],
        if_steps=[register_step],
        else_steps=[],
    )
    
    ### 12. Pipeline 수행
    pipeline = Pipeline(
        name="ts-prediction-informer-pipeline",
        parameters=[train_instance_param, train_count_param, model_approval_status],
        steps=[
            training_step,
            postprocessing_step,
            cond_step
        ],
    )

    return pipeline




작성한 pipeline.py와 함께 **pipelines** 폴더와 **codebuild-buildspec.yml** 파일을 SageMaker Project로 생성한 CodeCommit의 repository 안의 값과 대체합니다.

In [ ]:
repositoryName

In [ ]:
!cp -r ./pipelines ./$repositoryName/
!cp -r ./codebuild-buildspec.yml ./$repositoryName/codebuild-buildspec.yml


이후 SageMaker Studio에서 모든 소스코드를 commit한 다음 push를 하면 이후 자동으로 SageMaker Pipeline은 실행이 됩니다. 실행되는 모습은 SageMaker Pipeline에서 확인하거나 Codepipeline에서 확인하실 수 있습니다.
<p align="center">
<center><img src="./img/pipeline_codecommit.png" height="250" width="850" alt=""><center>
<br><br>
</p>
<p align="center">
<center><img src="./img/code_commit.png" height="250" width="750" alt=""><center>
<br><br>
</p>  

### 2. MLOps에서 활용할 Policy 설정하기

해당 HOL에서 구현할 아키텍처에 필요한 managed policy를 아래와 같이 정의합니다. Role을 별도 생성하셔도 되지만 HOL의 편의성을 위해 SageMaker Notebook/Studio와 동일한 Role에 policy를 추가하여 계속 활용합니다.

In [ ]:
role=get_execution_role()
base_role_name=role.split('/')[-1]

In [ ]:
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'
)
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AWSLambda_FullAccess'
)

### 3. SageMaker Studio의 설정값에서 model_package_group_name 가져오기
이 작업을 진행하기 이전에 SageMaker Studio에서 Project 생성이 필요합니다.

In [ ]:
sm_client = boto3.client("sagemaker")

project_name=sm_client.list_projects(
    SortBy='CreationTime',
    SortOrder='Descending')['ProjectSummaryList'][0]['ProjectName']
project_name

project_response=sm_client.describe_project(ProjectName=project_name)
model_package_group_name = project_response['ProjectName']+"-"+project_response['ProjectId']
model_package_group_name

### 4. Create Amazon EventBridge Rule

model registry에서 모델이 **Approved**되었을 때 이벤트 트리거를 만들기 위한 설정을 Amazon EventBridge Rule을 이용하여 설정합니다.

In [ ]:
event_client = boto3.client('events')

In [ ]:
eventpattern = json.dumps(
    {
      "source": ["aws.sagemaker"],
      "detail-type": ["SageMaker Model Package State Change"],
      "detail": {
        "ModelPackageGroupName": [f"{model_package_group_name}"],
        "ModelApprovalStatus": ["Approved"]
      }
    }
)

In [ ]:
time.sleep(7)
rule_name = 'informer_model_package_state'
event_rule = event_client.put_rule(
    Name=rule_name,
    EventPattern=eventpattern,
    State='ENABLED',
    Description='This is after the approval update for the Informer model',
)

### 5. Lambda function 생성

EventBridge 에서 Rule 만족하는 이벤트가 발생했을 때 실행되는 Lambda Function을 정의합니다. Lambda Function 은 테스트 데이터를 예측하는 Batch transform job을 수행하게 됩니다.


In [ ]:
!mkdir batch_transform

In [ ]:
%%writefile batch_transform/sm_batch_transform.py

import json
import boto3
import os
from time import strftime

from sagemaker.pytorch.model import PyTorchModel

def lambda_handler(event, context):
    """
    모델 레지스트리에서 최신 버전의 모델 승인 상태를 변경하는 람다 함수.
    """
    
#     try:
    ##############################################
    # 람다 함수는 Event Bridge의 패턴 정보를 event 개체를 통해서 받습니다.
    ##############################################   
    print(f"event : {event}")
    model_package_arn = event['detail']["ModelPackageArn"]
    model_package_group_name = event['detail']["ModelPackageGroupName"]
    print("model_package_arn: ", model_package_arn)      
    print("model_package_group_name: ", model_package_group_name)

    ### 환경 변수
    secret_name = os.environ["SECRET_NAME"]
    code_commit_repo = os.environ["CODE_COMMIT_REPO"]
    default_bucket = os.environ["DEFAULT_BUCKET"]
    role = os.environ["ROLE"] 

    secret=get_secret(secret_name)

    git_config = {'repo': code_commit_repo,
                  'branch': 'main',
                  'username': secret['username'],
                  'password': secret['password']}

    code_location = f's3://{default_bucket}/poc_informer/sm_codes'

    sess = boto3.Session()
    sm = sess.client('sagemaker')
    model_pkg = sm.describe_model_package(ModelPackageName=model_package_arn)

    model = PyTorchModel(
        entry_point='predictor.py',
        source_dir='Informer2020',
        git_config=git_config,
        code_location=code_location,
        model_data=model_pkg['InferenceSpecification']['Containers'][0]['ModelDataUrl'],
        role=role,
        framework_version="1.10",
        py_version="py38"
    )

    transformer= model.transformer(
        instance_count=1,
        instance_type='ml.m5.xlarge',
        assemble_with="Line",
        output_path=f"s3://{default_bucket}/poc_informer/batch_result",
        env={'default_bucket': default_bucket}
    )

    from time import strftime

    job_name=model_package_group_name+"-"+strftime("%m%d-%H%M%s")

    batch_transform = transformer.transform(
        data=f's3://{default_bucket}/dataset/ETDataset/ETT-small/ETTh1.csv',
        data_type='S3Prefix',
        content_type='text/csv',
        split_type='Line',
        job_name=f"tranform-{job_name}",
        wait=False
    )

    return_msg = f"Starting Batch Transform"

    ##############################################        
    # 람다 함수의 리턴 정보를 구성하고 리턴 합니다.
    ##############################################        

    return {
        "statusCode": 200,
        "body": json.dumps(return_msg),
        "other_key": "example_value",
    }

#     except BaseException as error:
#         return_msg = f"There is no model_package_group_name {model_package_group_name}"                
#         error_msg = f"An exception occurred: {error}"
#         print(error_msg)    
#         return {
#             "statusCode": 500,
#             "body": json.dumps(return_msg),
#             "other_key": "example_value",
#         }        
        
def get_secret(secret_name):
    secret = {}
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager'
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )
        
    if 'SecretString' in get_secret_value_response:
        secret = get_secret_value_response['SecretString']
        secret = json.loads(secret)
    else:
        print("secret is not defined. Checking the Secrets Manager")

    return secret


In [ ]:
%%writefile batch_transform/Dockerfile

# Define function directory
ARG FUNCTION_DIR="/function"

FROM python:buster as build-image

# Install aws-lambda-cpp build dependencies
RUN apt-get update && \
  apt-get install -y \
  g++ \
  make \
  cmake \
  unzip \
  git \
  libcurl4-openssl-dev

# Include global arg in this stage of the build
ARG FUNCTION_DIR
# Create function directory
RUN mkdir -p ${FUNCTION_DIR}

# Copy function code
COPY sm_batch_transform.py ${FUNCTION_DIR}
# COPY git_lambda ${FUNCTION_DIR}/git_lambda
# COPY yolov5 ${FUNCTION_DIR}/yolov5

# Install the runtime interface client
RUN pip install \
        --target ${FUNCTION_DIR} \
        awslambdaric sagemaker smdebug sagemaker-experiments

# Multi-stage build: grab a fresh copy of the base image
FROM python:buster

# Include global arg in this stage of the build
ARG FUNCTION_DIR
# Set working directory to function root directory
WORKDIR ${FUNCTION_DIR}

# Copy in the build image dependencies
COPY --from=build-image ${FUNCTION_DIR} ${FUNCTION_DIR}

ENTRYPOINT [ "/usr/local/bin/python", "-m", "awslambdaric" ]
CMD [ "sm_batch_transform.lambda_handler" ]

In [ ]:
%%bash
cd ./batch_transform
echo $(pwd)
container_name=lambda-informer-batchtransform
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:1.0"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${container_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${container_name}" > /dev/null
fi

# # Get the login command from ECR and execute it directly
# $(aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin "763104351884.dkr.ecr.us-west-2.amazonaws.com")

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build -f Dockerfile -t ${fullname} .
# docker tag ${container_name} ${fullname}

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)
docker push ${fullname}

In [ ]:
my_session = boto3.session.Session()
region = my_session.region_name

repo_name=f"{account_id}.dkr.ecr.{region}.amazonaws.com/lambda-informer-batchtransform:1.0"
repo_name

In [ ]:
lambda_client = boto3.client('lambda')

In [ ]:
lambda_trust_policy=json.dumps({
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
})

In [ ]:
role_name='lambda-assume-role_'+ strftime("%m%d-%H%M%s")
try:
    for role_list in iam_client.list_roles()['Roles']:
        pre_role_name = role_list['RoleName']
        if pre_role_name.split("_")[0] in ['lambda-assume-role']:
            iam_client.detach_role_policy(
                RoleName=pre_role_name,
                PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
            )
            iam_client.detach_role_policy(
                RoleName=pre_role_name,
                PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
            )
            iam_client.delete_role(RoleName=pre_role_name)
except:
    pass
finally:
    lambda_role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=lambda_trust_policy
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/SecretsManagerReadWrite'
    )
    time.sleep(10)

In [ ]:
### CodeCommit의 Credentials이 저장된 secret_name 사용이 필요합니다.
sec_client = boto3.client('secretsmanager')
secret_name = sec_client.list_secrets(SortOrder='desc')['SecretList'][0]['ARN']  

In [ ]:
lambda_name='informer-batch-transform'
try:
    lambda_client.delete_function(FunctionName=lambda_name)
except:
    pass
finally:
    lambda_response = lambda_client.create_function(
        FunctionName=lambda_name,
        Role=lambda_role['Role']['Arn'],
        Code={
            'ImageUri': repo_name
        },
        PackageType='Image',
        Description='batch transform of the latest version-based infomer model',
        Timeout=600,
        MemorySize=512,
        Environment={
          'Variables': {
              "SECRET_NAME" : secret_name,
              "CODE_COMMIT_REPO" : code_commit_repo,
              "DEFAULT_BUCKET" : default_bucket,
              "ROLE" : role
          }
      }
    )    

In [ ]:
lambda_permission_response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='InvokeLambdaFunction',
    Action='lambda:InvokeFunction',
    Principal="events.amazonaws.com",
    SourceArn=event_rule['RuleArn'],
)

Amazon EventBridge에 위에서 생성한 Lambda function을 타켓으로 설정합니다.

In [ ]:
event_client.put_targets(
    Rule=rule_name,
    Targets=[
        {
            'Id': 'Target0',
            'Arn': lambda_response['FunctionArn']
        }
    ]
)

### 6. Studio에서 실행하기


SageMaker Pipelines의 실행 과정은 SageMaker Studio를 보면 아래와 같이 UI로 확인이 가능합니다.
<p align="center">
<center><img src="./img/mlops_exe.png" height="350" width="750" alt=""><center>
<br><br>
</p>  
    
SageMaker Pipeline이 수행된 다음에는 학습된 model은 model registry에 버전별로 등록이 됩니다. 등록된 model을 **Approval**가 하게되면 앞서 설정한 EventBridge가 해당 이벤트를 rule에서 판단하게 되고 이후 테스트 데이터를 예측하는 Batch transform job을 수행하는 Lambda가 시작됩니다. 
<p align="center">
<center><img src="./img/model_registry.png" height="350" width="750" alt=""><center>
<br><br>
</p>      
    
    
    

### 7. QuickSight 생성하기

AWS 콘솔에서 QuickSight 서비스를 생성합니다. 기본 설정에서 하단의 continue 버튼을 클릭합니다.
<p align="center">
<center><img src="./img/quicksight_start.png" height="250" width="550" alt=""><center>
<br><br>
</p>  
 
- Region은 편의성을 위해 현재 사용 중인 리전을 선택합니다. 
- 다음 Account Name은 unique한 이름으로 설정합니다. 
- 이메일 주소를 넣습니다.
- Amazon S3를 선택한 다음 default_bucket 버킷명을 찾아서 선택하고 Finish 버튼을 클릭합니다.

In [ ]:
print(default_bucket)

    
<p align="center">
<center><img src="./img/quicksight_setting.png" height="750" width="1050" alt=""><center>
<br><br>
</p>  

 

### 8. QuickSight에서 사용할 manifest_file 생성하기

quicksight에서 사용할 manifest_file 생성합니다.

In [ ]:
manifest_file = {
                "fileLocations": [
                    {
                        "URIPrefixes": [
                            f"s3://{default_bucket}/poc_informer/batch_result/"
                        ]
                    }
                ],
                "globalUploadSettings": {
                    "format": "CSV",
                    "delimiter": ",",
                    "textqualifier": "\"",
                    "containsHeader": "true"
                }
            }

In [ ]:
with open("./quicksight/manifest_file.json", 'w', encoding="utf-8") as f:
    json.dump(manifest_file, f, indent="\t")

   
왼쪽 메뉴에서 **Datasets**를 선택한 다음 오른쪽 상단의 **New dataset**을 선택합니다. 이후 **S3**를 선택하면 아래와 같이 팝업 창이 뜨고, 원하는 이름으로 **data source name**을 설정한 다음, **upload a manifest file**에는 위에 copy한 S3 주소를 입력합니다.

In [ ]:
!aws s3 sync ./quicksight/prediction_result s3://$default_bucket/poc_informer/batch_result/ --quiet
!aws s3 cp ./quicksight/manifest_file.json s3://$default_bucket/poc_informer/quick_sight/ --quiet
print(f"s3://{default_bucket}/poc_informer/quick_sight/manifest_file.json")

<p align="center">
<center><img src="./img/quicksight_dataset.png" height="750" width="1050" alt=""><center>
<br><br>
</p>  

앞에서 생성한 dataset에서 **visualize**를 선택하면 아래와 같이 그래프 생성이 가능합니다. **date**는 X axis로, **OT (Ground Truth)**, **Prediction**는 Value로 선택한 다음, date의 간격을 **Hour**로 변경해 줍니다.

<p align="center">
<center><img src="./img/quicksight_vis.png" height="750" width="1050" alt=""><center>
<br><br>
</p>  

QuickSight에 관련된 자세한 실습은 [QuickSight Workshop](https://learnquicksight.workshop.aws/en/author-workshop/0.prerequisites.html) 에서 수행해 보시기 바랍니다.

### 9. quicksight spice dataset을 refresh하기 (Optional)

모델이 새롭게 업데이트된 다음 다시 예측을 수행하게 되면 새롭게 생성된 결과 CSV 파일을 다시 업데이트하여 QuickSight Figure을 업데이트해야 합니다.

In [ ]:
qs_client = boto3.client("quicksight")

In [ ]:
res = qs_client.list_data_sets(AwsAccountId=account_id)

# filter out your datasets using a prefix. All my datasets have chicago_crimes as their prefix
datasets_ids = [summary["DataSetId"] for summary in res["DataSetSummaries"]]
ingestion_ids = []

for dataset_id in datasets_ids:
    try:
        ingestion_id = str(calendar.timegm(time.gmtime()))
#         ingestion_id = str(uuid.uuid4())
        qs_client.create_ingestion(DataSetId=dataset_id, IngestionId=ingestion_id,
                                             AwsAccountId=account_id)
        ingestion_ids.append(ingestion_id)
    except Exception as e:
        print(e)
        pass

for ingestion_id, dataset_id in zip(ingestion_ids, datasets_ids):
    while True:
        response = qs_client.describe_ingestion(DataSetId=dataset_id,
                                             IngestionId=ingestion_id,
                                             AwsAccountId=account_id)
        if response['Ingestion']['IngestionStatus'] in ('INITIALIZED', 'QUEUED', 'RUNNING'):
            time.sleep(5)     #change sleep time according to your dataset size
        elif response['Ingestion']['IngestionStatus'] == 'COMPLETED':
            print("refresh completed. RowsIngested {0}, RowsDropped {1}, IngestionTimeInSeconds {2}, IngestionSizeInBytes {3}".format(
                response['Ingestion']['RowInfo']['RowsIngested'],
                response['Ingestion']['RowInfo']['RowsDropped'],
                response['Ingestion']['IngestionTimeInSeconds'],
                response['Ingestion']['IngestionSizeInBytes']))
            break
        else:
            print("refresh failed for {0}! - status {1}".format(dataset_id, response['Ingestion']['IngestionStatus']))
            break